In [68]:
import json
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.model_selection import  train_test_split
from sklearn.linear_model import LinearRegression,Lasso,LogisticRegression,SGDRegressor,Ridge
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.model_selection import  train_test_split
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier,GradientBoostingClassifier,GradientBoostingRegressor
from sklearn.metrics import mean_squared_error,accuracy_score
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings("ignore")

### **Reading the neccesary data required**

In [27]:
file=open('/content/algoparams_from_ui.json')
model_info=json.load(file)
print(model_info)

{'session_name': 'test', 'session_description': 'test', 'design_state_data': {'session_info': {'project_id': '1', 'experiment_id': 'kkkk-11', 'dataset': 'iris_modified.csv', 'session_name': 'test', 'session_description': 'test'}, 'target': {'prediction_type': 'Regression', 'target': 'petal_width', 'type': 'regression', 'partitioning': True}, 'train': {'policy': 'Split the dataset', 'time_variable': 'sepal_length', 'sampling_method': 'No sampling(whole data)', 'split': 'Randomly', 'k_fold': False, 'train_ratio': 0, 'random_seed': 0}, 'metrics': {'optomize_model_hyperparameters_for': 'AUC', 'optimize_threshold_for': 'F1 Score', 'compute_lift_at': 0, 'cost_matrix_gain_for_true_prediction_true_result': 1, 'cost_matrix_gain_for_true_prediction_false_result': 0, 'cost_matrix_gain_for_false_prediction_true_result': 0, 'cost_matrix_gain_for_false_prediction_false_result': 0}, 'feature_handling': {'sepal_length': {'feature_name': 'sepal_length', 'is_selected': True, 'feature_variable_type': 'nu

In [28]:
df=pd.read_csv("/content/iris.csv")
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [29]:
info=['train','target','metrics']
for topic in info:
  print(f'{topic}:',model_info['design_state_data'][topic])

train: {'policy': 'Split the dataset', 'time_variable': 'sepal_length', 'sampling_method': 'No sampling(whole data)', 'split': 'Randomly', 'k_fold': False, 'train_ratio': 0, 'random_seed': 0}
target: {'prediction_type': 'Regression', 'target': 'petal_width', 'type': 'regression', 'partitioning': True}
metrics: {'optomize_model_hyperparameters_for': 'AUC', 'optimize_threshold_for': 'F1 Score', 'compute_lift_at': 0, 'cost_matrix_gain_for_true_prediction_true_result': 1, 'cost_matrix_gain_for_true_prediction_false_result': 0, 'cost_matrix_gain_for_false_prediction_true_result': 0, 'cost_matrix_gain_for_false_prediction_false_result': 0}


In [30]:
info=['sepal_length','sepal_width','petal_length','petal_width','species']
for topic in info:
  print(model_info['design_state_data']["feature_handling"][topic])

{'feature_name': 'sepal_length', 'is_selected': True, 'feature_variable_type': 'numerical', 'feature_details': {'numerical_handling': 'Keep as regular numerical feature', 'rescaling': 'No rescaling', 'make_derived_feats': False, 'missing_values': 'Impute', 'impute_with': 'Average of values', 'impute_value': 0}}
{'feature_name': 'sepal_width', 'is_selected': True, 'feature_variable_type': 'numerical', 'feature_details': {'numerical_handling': 'Keep as regular numerical feature', 'rescaling': 'No rescaling', 'make_derived_feats': False, 'missing_values': 'Impute', 'impute_with': 'custom', 'impute_value': -1}}
{'feature_name': 'petal_length', 'is_selected': True, 'feature_variable_type': 'numerical', 'feature_details': {'numerical_handling': 'Keep as regular numerical feature', 'rescaling': 'No rescaling', 'make_derived_feats': False, 'missing_values': 'Impute', 'impute_with': 'Average of values', 'impute_value': 0}}
{'feature_name': 'petal_width', 'is_selected': True, 'feature_variable_t

### **Preparing data for model**

In [31]:
df.dtypes

sepal_length    float64
sepal_width     float64
petal_length    float64
petal_width     float64
species          object
dtype: object

In [32]:
#performing one hot encoding on species column 
df=pd.get_dummies(df)
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species_Iris-setosa,species_Iris-versicolor,species_Iris-virginica
0,5.1,3.5,1.4,0.2,1,0,0
1,4.9,3.0,1.4,0.2,1,0,0
2,4.7,3.2,1.3,0.2,1,0,0
3,4.6,3.1,1.5,0.2,1,0,0
4,5.0,3.6,1.4,0.2,1,0,0


In [33]:
#imputing with average value
imputer=SimpleImputer(strategy='mean',missing_values=np.nan)
df[['sepal_length','petal_length']]=imputer.fit_transform(df[['sepal_length','petal_length']])

In [34]:
#custom impute 
sepal_width_imputer=SimpleImputer(strategy ='constant', fill_value=model_info['design_state_data']["feature_handling"]['sepal_width']['feature_details']['impute_value'])
petal_width_imputer=SimpleImputer(strategy ='constant', fill_value=model_info['design_state_data']["feature_handling"]['petal_width']['feature_details']['impute_value'])

In [35]:
#creating the dependent and independent frames
x=df.drop([model_info['design_state_data']['target']['target']],axis=1)
y=df[model_info['design_state_data']['target']['target']]
print(x.head(),'\n\n',y.head())

   sepal_length  sepal_width  petal_length  species_Iris-setosa  \
0           5.1          3.5           1.4                    1   
1           4.9          3.0           1.4                    1   
2           4.7          3.2           1.3                    1   
3           4.6          3.1           1.5                    1   
4           5.0          3.6           1.4                    1   

   species_Iris-versicolor  species_Iris-virginica  
0                        0                       0  
1                        0                       0  
2                        0                       0  
3                        0                       0  
4                        0                       0   

 0    0.2
1    0.2
2    0.2
3    0.2
4    0.2
Name: petal_width, dtype: float64


**Creating object of each model and fitting the model and calculating mean square error**

**I have written code for few machine learning algorithm then we can add more algorithm**

**In last I will create a function which will take df, algoparams_from_ui and provide ouput**

In [ ]:
#creating all the models specified 
model_info['design_state_data']['algorithms'].keys()

dict_keys(['RandomForestClassifier', 'RandomForestRegressor', 'GBTClassifier', 'GBTRegressor', 'LinearRegression', 'LogisticRegression', 'RidgeRegression', 'LassoRegression', 'ElasticNetRegression', 'xg_boost', 'DecisionTreeRegressor', 'DecisionTreeClassifier', 'SVM', 'SGD', 'KNN', 'extra_random_trees', 'neural_network'])

In [36]:
#performing train test split
from sklearn.model_selection import  train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,shuffle=True)

In [47]:
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier,GradientBoostingClassifier,GradientBoostingRegressor
from sklearn.metrics import mean_squared_error,accuracy_score

#in tree reduction most of the parameter are already in place with model only max_feature was not present so coded for that 
try:
  no_of_feature=float(model_info['design_state_data']["feature_reduction"]['num_of_features_to_keep'])
  n_features=3#total number of features
  max_feature=no_of_feature/n_features
  if max_feature>1.0:
    max_feature=1.0
except Exception as e:
  max_feature=1.0

rf_model=RandomForestRegressor(n_estimators=model_info['design_state_data']['algorithms']['RandomForestRegressor']["max_trees"], criterion='squared_error', 
                      max_depth=model_info['design_state_data']['algorithms']['RandomForestRegressor']["max_depth"], min_samples_split=2, 
                      min_samples_leaf=model_info['design_state_data']['algorithms']['RandomForestRegressor']["min_samples_per_leaf_max_value"], 
                      min_weight_fraction_leaf=0.0, max_features=max_feature, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, bootstrap=True, oob_score=False, n_jobs=None, random_state=1, verbose=0, 
                      warm_start=False, ccp_alpha=0.0, max_samples=None)


param_grid={'n_estimators':np.arange(model_info['design_state_data']['algorithms']['RandomForestRegressor']["min_trees"],model_info['design_state_data']['algorithms']['RandomForestRegressor']["max_trees"]+1),
            'max_depth':np.arange(model_info['design_state_data']['algorithms']['RandomForestRegressor']["min_depth"],model_info['design_state_data']['algorithms']['RandomForestRegressor']["max_depth"]+1),
            'min_samples_leaf':np.arange(model_info['design_state_data']['algorithms']['RandomForestRegressor']["min_samples_per_leaf_min_value"],model_info['design_state_data']['algorithms']['RandomForestRegressor']["min_samples_per_leaf_max_value"]+1)}

cv_rf_model=GridSearchCV(rf_model, param_grid, scoring=None, n_jobs=model_info['design_state_data']["hyperparameters"]["parallelism"], refit=True, 
                         cv=model_info['design_state_data']["hyperparameters"]["num_of_folds"],
                         verbose=0, pre_dispatch='2*n_jobs', error_score=np.nan, return_train_score=False)

cv_rf_model.fit(xtrain,ytrain)
pred_y=cv_rf_model.predict(xtest)
print(f"Random Forest Regressor  mean squared error is {round(mean_squared_error(ytest,pred_y),2)}")

Random Forest Regressor  mean squared error is 0.02


In [60]:
rf_model=RandomForestClassifier(n_estimators=model_info['design_state_data']['algorithms']['RandomForestClassifier']["max_trees"], criterion='gini', 
                       max_depth=model_info['design_state_data']['algorithms']['RandomForestClassifier']["max_depth"],
                       min_samples_split=2, min_samples_leaf=model_info['design_state_data']['algorithms']['RandomForestClassifier']["min_samples_per_leaf_max_value"],
                       min_weight_fraction_leaf=0.0, max_features=max_feature, max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True,
                       oob_score=False, n_jobs=None, random_state=1, verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)

param_grid={'n_estimators':np.arange(model_info['design_state_data']['algorithms']['RandomForestClassifier']["min_trees"],model_info['design_state_data']['algorithms']['RandomForestClassifier']["max_trees"]+1),
            'max_depth':np.arange(model_info['design_state_data']['algorithms']['RandomForestClassifier']["min_depth"],model_info['design_state_data']['algorithms']['RandomForestClassifier']["max_depth"]+1),
            'min_samples_leaf':np.arange(model_info['design_state_data']['algorithms']['RandomForestClassifier']["min_samples_per_leaf_min_value"],model_info['design_state_data']['algorithms']['RandomForestClassifier']["min_samples_per_leaf_max_value"]+1)}

cv_rf_model=GridSearchCV(rf_model, param_grid, scoring=None, n_jobs=model_info['design_state_data']["hyperparameters"]["parallelism"], refit=True, 
                         cv=model_info['design_state_data']["hyperparameters"]["num_of_folds"],
                         verbose=0, pre_dispatch='2*n_jobs', error_score=np.nan, return_train_score=False)


rf_model.fit(xtrain,ytrain)
pred_y=rf_model.predict(xtest)
print(f"Random Forest classifier  accuracy is {accuracy_score(ytest,pred_y)}")

In [63]:
gbc_model=GradientBoostingClassifier(loss='log_loss', learning_rate=0.1, n_estimators=5, subsample=1.0, criterion='friedman_mse', min_samples_split=2, 
                                            min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=model_info['design_state_data']['algorithms']['GBTClassifier']["max_depth"],
                                            min_impurity_decrease=0.0, init=None, random_state=1, 
                                            max_features=max_feature, verbose=0, max_leaf_nodes=None, warm_start=False, validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)

param_grid={'subsample':np.arange(model_info['design_state_data']['algorithms']['GBTClassifier']["min_subsample"],model_info['design_state_data']['algorithms']['GBTClassifier']["max_subsample"]+1),
            'max_depth':np.arange(model_info['design_state_data']['algorithms']['GBTClassifier']["min_depth"],model_info['design_state_data']['algorithms']['GBTClassifier']["max_depth"]+1),
            'n_iter_no_change':np.arange(model_info['design_state_data']['algorithms']['GBTClassifier']["min_iter"],model_info['design_state_data']['algorithms']['GBTClassifier']["max_iter"]+1)}

cv_gb_model=GridSearchCV(gbc_model, param_grid, scoring=None, n_jobs=model_info['design_state_data']["hyperparameters"]["parallelism"], refit=True, 
                         cv=model_info['design_state_data']["hyperparameters"]["num_of_folds"],
                         verbose=0, pre_dispatch='2*n_jobs', error_score=np.nan, return_train_score=False)

cv_gb_model.fit(xtrain,ytrain)
pred_y=cv_gb_model.predict(xtest)
print(f"Gradient Boosted classifier  accuracy is {accuracy_score(ytest,pred_y)}")

In [69]:
gbr_model=GradientBoostingRegressor(loss='squared_error', learning_rate=0.1, n_estimators=10, subsample=0.5, criterion='friedman_mse', min_samples_split=2,
                                            min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=model_info['design_state_data']['algorithms']['GBTRegressor']["max_depth"], min_impurity_decrease=0.0, init=None, random_state=1,
                                           max_features=None, alpha=0.9, verbose=0, max_leaf_nodes=None, warm_start=False, validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)


param_grid={'subsample':np.arange(model_info['design_state_data']['algorithms']['GBTRegressor']["min_subsample"],model_info['design_state_data']['algorithms']['GBTRegressor']["max_subsample"]+1),
            'max_depth':np.arange(model_info['design_state_data']['algorithms']['GBTRegressor']["min_depth"],model_info['design_state_data']['algorithms']['GBTRegressor']["max_depth"]+1),
            'n_iter_no_change':np.arange(model_info['design_state_data']['algorithms']['GBTRegressor']["min_iter"],model_info['design_state_data']['algorithms']['GBTRegressor']["max_iter"]+1)}

cv_gb_model=GridSearchCV(gbr_model, param_grid, scoring=None, n_jobs=model_info['design_state_data']["hyperparameters"]["parallelism"], refit=True, 
                         cv=model_info['design_state_data']["hyperparameters"]["num_of_folds"],
                         verbose=0, pre_dispatch='2*n_jobs', error_score=np.nan, return_train_score=False)

cv_gb_model.fit(xtrain,ytrain)
pred_y=cv_gb_model.predict(xtest)
print(f"Gradient Boosted Regressor  mean squared error is {round(mean_squared_error(ytest,pred_y),2)}")

Gradient Boosted Regressor  mean squared error is 0.09


In [70]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression(fit_intercept=True, normalize='deprecated', copy_X=True, n_jobs=None, positive=False)
lr.fit(xtrain,ytrain)
pred_y=lr.predict(xtest)
print(f"Linear Regression  mean squared error is {round(mean_squared_error(ytest,pred_y),2)}")

Linear Regression  mean squared error is 0.02


In [71]:
from sklearn.linear_model import LogisticRegression
logr=LogisticRegression(penalty='l2',dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=1,
                   solver='lbfgs', max_iter=model_info['design_state_data']['algorithms']['LogisticRegression']["max_iter"],
                    multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)

param_grid={'max_iter':np.arange(model_info['design_state_data']['algorithms']['LogisticRegression']["min_iter"],model_info['design_state_data']['algorithms']['LogisticRegression']["max_iter"]+1),
            }

cv_logr_model=GridSearchCV(logr, param_grid, scoring=None, n_jobs=model_info['design_state_data']["hyperparameters"]["parallelism"], refit=True, 
                         cv=model_info['design_state_data']["hyperparameters"]["num_of_folds"],
                         verbose=0, pre_dispatch='2*n_jobs', error_score=np.nan, return_train_score=False)

cv_logr_model.fit(xtrain,ytrain)
pred_y=cv_logr_model.predict(xtest)
print(f"Logistic Regression  accuracy is {accuracy_score(ytest,pred_y)}")

In [ ]:
model_info['design_state_data']['algorithms']['SGD']

{'alpha_value': [79, 56],
 'is_selected': False,
 'max_iterations': False,
 'model_name': 'Stochastic Gradient Descent',
 'parallelism': 1,
 'tolerance': 56,
 'use_elastic_net_regularization': True,
 'use_l1_regularization': 'on',
 'use_l2_regularization': 'on',
 'use_logistics': True,
 'use_modified_hubber_loss': False}

In [73]:
from sklearn.linear_model import Ridge
rr=Ridge(alpha=1.0, fit_intercept=True, normalize='deprecated', copy_X=True, max_iter=None, 
      tol=0.001, solver='auto', positive=False, random_state=None)

param_grid={'max_iter':np.arange(model_info['design_state_data']['algorithms']["RidgeRegression"]["min_iter"],model_info['design_state_data']['algorithms']["RidgeRegression"]["max_iter"]+1),
            }

cv_rr_model=GridSearchCV(rr, param_grid, scoring=None, n_jobs=model_info['design_state_data']["hyperparameters"]["parallelism"], refit=True, 
                         cv=model_info['design_state_data']["hyperparameters"]["num_of_folds"],
                         verbose=0, pre_dispatch='2*n_jobs', error_score=np.nan, return_train_score=False)

cv_rr_model.fit(xtrain,ytrain)
pred_y=cv_rr_model.predict(xtest)

print(f"Ridge Regressor  mean squared error is {round(mean_squared_error(ytest,pred_y),2)}")

Ridge Regressor  mean squared error is 0.02


In [75]:
from sklearn.linear_model import Lasso
lr=Lasso(alpha=1.0,fit_intercept=True, normalize='deprecated', precompute=False, copy_X=True, max_iter=1000, tol=0.0001, 
                        warm_start=False, positive=False, random_state=None, selection='cyclic')

param_grid={'max_iter':np.arange(model_info['design_state_data']['algorithms']["LassoRegression"]["min_iter"],model_info['design_state_data']['algorithms']["LassoRegression"]["max_iter"]+1),
            }


cv_lr_model=GridSearchCV(lr, param_grid, scoring=None, n_jobs=model_info['design_state_data']["hyperparameters"]["parallelism"], refit=True, 
                         cv=model_info['design_state_data']["hyperparameters"]["num_of_folds"],
                         verbose=0, pre_dispatch='2*n_jobs', error_score=np.nan, return_train_score=False)

cv_lr_model.fit(xtrain,ytrain)
pred_y=cv_lr_model.predict(xtest)
print(f"Lasso Regressor  mean squared error is {round(mean_squared_error(ytest,pred_y),2)}")

Lasso Regressor  mean squared error is 0.36


In [79]:
model_info['design_state_data']['algorithms']["DecisionTreeRegressor"]["use_gini"]

False

In [82]:
from sklearn.tree import DecisionTreeRegressor


dtr=DecisionTreeRegressor(criterion='squared_error', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, 
                          min_weight_fraction_leaf=0.0, max_features=max_feature, random_state=1, max_leaf_nodes=None, 
                          min_impurity_decrease=0.0, ccp_alpha=0.0)


param_grid={'max_depth':np.arange(model_info['design_state_data']['algorithms']["DecisionTreeRegressor"]["min_depth"],model_info['design_state_data']['algorithms']["DecisionTreeRegressor"]["max_depth"]+1)}

cv_dtr_model=GridSearchCV(dtr, param_grid, scoring=None, n_jobs=model_info['design_state_data']["hyperparameters"]["parallelism"], refit=True, 
                         cv=model_info['design_state_data']["hyperparameters"]["num_of_folds"],
                         verbose=0, pre_dispatch='2*n_jobs', error_score=np.nan, return_train_score=False)

cv_dtr_model.fit(xtrain,ytrain)
pred_y=cv_dtr_model.predict(xtest)
print(f"Decision Tree Regression  mean square error is {round(mean_squared_error(ytest,pred_y),2)}")

Decision Tree Regression  mean square error is 0.04


In [84]:
from sklearn.tree import DecisionTreeClassifier

if model_info['design_state_data']['algorithms']["DecisionTreeClassifier"]["use_gini"]==False:
  criterion='entropy'
else:
  criterion='gini'

dtc=DecisionTreeClassifier(criterion=criterion, splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1,
                                    min_weight_fraction_leaf=0.0, max_features=max_feature, random_state=1, max_leaf_nodes=None, min_impurity_decrease=0.0, 
                                    class_weight=None, ccp_alpha=0.0)

param_grid={'max_depth':np.arange(model_info['design_state_data']['algorithms']["DecisionTreeClassifier"]["min_depth"],model_info['design_state_data']['algorithms']["DecisionTreeClassifier"]["max_depth"]+1)}

cv_dtc_model=GridSearchCV(dtc, param_grid, scoring=None, n_jobs=model_info['design_state_data']["hyperparameters"]["parallelism"], refit=True, 
                         cv=model_info['design_state_data']["hyperparameters"]["num_of_folds"],
                         verbose=0, pre_dispatch='2*n_jobs', error_score=np.nan, return_train_score=False)

cv_dtc_model.fit(xtrain,ytrain)
pred_y=cv_dtc_model.predict(xtest)
print(f"Decision Tree classifier  accuracy is {accuracy_score(ytest,pred_y)}")

In [ ]:
from sklearn.linear_model import SGDRegressor
sgd=SGDRegressor(loss='squared_error',penalty='l2', alpha=model_info['design_state_data']['algorithms']['SGD']["alpha_value"][0], l1_ratio=0.15,
                                  fit_intercept=True, max_iter=1000, tol=model_info['design_state_data']['algorithms']['SGD']["tolerance"], 
                                  shuffle=True, verbose=0, epsilon=0.1, random_state=None, learning_rate='invscaling', eta0=0.01, power_t=0.25, 
                                  early_stopping=False, validation_fraction=0.1, n_iter_no_change=5, warm_start=False, average=False)

sgd.fit(xtrain,ytrain)
pred_y=sgd.predict(xtest)
print(f"SGD Regression  mean squared error is {round(mean_squared_error(ytest,pred_y),2)}")

SGD Regression  mean squared error is 0.88


In [ ]:
#finding out which model to run
model_instance={}
for name in model_info['design_state_data']['algorithms'].keys():
  model_instance[name]=model_info['design_state_data']['algorithms'][name]['is_selected']
model_instance

{'DecisionTreeClassifier': False,
 'DecisionTreeRegressor': False,
 'ElasticNetRegression': False,
 'GBTClassifier': False,
 'GBTRegressor': False,
 'KNN': False,
 'LassoRegression': False,
 'LinearRegression': False,
 'LogisticRegression': False,
 'RandomForestClassifier': False,
 'RandomForestRegressor': True,
 'RidgeRegression': False,
 'SGD': False,
 'SVM': False,
 'extra_random_trees': False,
 'neural_network': False,
 'xg_boost': False}

## **Version 2.0**

##**A few points to fix and try again:**

 

1. Your code should be generic enough to handle different JSON files
You are skipping hyper params and GridSearch as you do not fit the model on the CV object



> **Performed hyperparameter tuning using grid search**



2. It is ok to have the exploratory code on top, but what we are looking for in a successful test submitted back is bucketed into four parts:
Code that can parse the JSON properly

> **Tried to parse the json file available information**

3. Do the Feature handling properly. Keep in mind that impute_with can do Average of Values or Mean of Values, and you should have switch conditions to tackle that

> **Used SimpleImpute for all the condition**

4. Do Feature Reduction – if there is no reduction, do nothing; if the reduction is tree-based, then do that
Models Built

> **Done Feature Reduction(Most of the parameter of feature reduction were already intact, Just added the max_feature parameter to all tree based model)**

### **Created function for prediction using the above code**

I was unable to find most of parameter specified in the json file for model.

In [89]:
#df:read the iris file 
#model_info: read the algoparagms_from_ui.json

df=pd.read_csv("/content/iris.csv")
file=open('/content/algoparams_from_ui.json')
model_info=json.load(file)


def model_prediction(df,model_info):
  #preprocessing and making data ready for model
  df=pd.get_dummies(df)
  imputer=SimpleImputer(strategy='mean',missing_values=np.nan)
  sepal_width_imputer=SimpleImputer(strategy ='constant', fill_value=model_info['design_state_data']["feature_handling"]['sepal_width']['feature_details']['impute_value'])
  petal_width_imputer=SimpleImputer(strategy ='constant', fill_value=model_info['design_state_data']["feature_handling"]['petal_width']['feature_details']['impute_value'])
  
  df[['sepal_length','petal_length']]=imputer.fit_transform(df[['sepal_length','petal_length']])
  df["sepal_width"]=sepal_width_imputer.fit_transform(np.array(df["sepal_width"]).reshape(-1, 1))
  df["petal_width"]=petal_width_imputer.fit_transform(np.array(df["petal_width"]).reshape(-1,1))

  xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,shuffle=True)

  #in tree reduction most of the parameter are already in place with model only max_feature was not present so coded for that 
  try:
    no_of_feature=float(model_info['design_state_data']["feature_reduction"]['num_of_features_to_keep'])
    n_features=3#total number of features
    max_feature=no_of_feature/n_features
    if max_feature>1.0:
      max_feature=1.0
  except Exception as e:
    max_feature=1.0



  #finding out which model to run
  model_instance={}
  for name in model_info['design_state_data']['algorithms'].keys():
    model_instance[name]=model_info['design_state_data']['algorithms'][name]['is_selected']

  if model_instance['RandomForestRegressor']:
      rf_model=RandomForestRegressor(n_estimators=model_info['design_state_data']['algorithms']['RandomForestRegressor']["max_trees"], criterion='squared_error', 
                      max_depth=model_info['design_state_data']['algorithms']['RandomForestRegressor']["max_depth"], min_samples_split=2, 
                      min_samples_leaf=model_info['design_state_data']['algorithms']['RandomForestRegressor']["min_samples_per_leaf_max_value"], 
                      min_weight_fraction_leaf=0.0, max_features=max_feature, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, bootstrap=True, oob_score=False, n_jobs=None, random_state=1, verbose=0, 
                      warm_start=False, ccp_alpha=0.0, max_samples=None)


      param_grid={'n_estimators':np.arange(model_info['design_state_data']['algorithms']['RandomForestRegressor']["min_trees"],model_info['design_state_data']['algorithms']['RandomForestRegressor']["max_trees"]+1),
                  'max_depth':np.arange(model_info['design_state_data']['algorithms']['RandomForestRegressor']["min_depth"],model_info['design_state_data']['algorithms']['RandomForestRegressor']["max_depth"]+1),
                  'min_samples_leaf':np.arange(model_info['design_state_data']['algorithms']['RandomForestRegressor']["min_samples_per_leaf_min_value"],model_info['design_state_data']['algorithms']['RandomForestRegressor']["min_samples_per_leaf_max_value"]+1)}

      cv_rf_model=GridSearchCV(rf_model, param_grid, scoring=None, n_jobs=model_info['design_state_data']["hyperparameters"]["parallelism"], refit=True, 
                              cv=model_info['design_state_data']["hyperparameters"]["num_of_folds"],
                              verbose=0, pre_dispatch='2*n_jobs', error_score=np.nan, return_train_score=False)

      cv_rf_model.fit(xtrain,ytrain)
      pred_y=cv_rf_model.predict(xtest)
      print(f"Random Forest Regressor  mean squared error is {round(mean_squared_error(ytest,pred_y),2)}")  
  
  elif model_instance['RandomForestClassifier']:
      rf_model=RandomForestClassifier(n_estimators=model_info['design_state_data']['algorithms']['RandomForestClassifier']["max_trees"], criterion='gini', 
                            max_depth=model_info['design_state_data']['algorithms']['RandomForestClassifier']["max_depth"],
                            min_samples_split=2, min_samples_leaf=model_info['design_state_data']['algorithms']['RandomForestClassifier']["min_samples_per_leaf_max_value"],
                            min_weight_fraction_leaf=0.0, max_features=max_feature, max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True,
                            oob_score=False, n_jobs=None, random_state=1, verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)

      param_grid={'n_estimators':np.arange(model_info['design_state_data']['algorithms']['RandomForestClassifier']["min_trees"],model_info['design_state_data']['algorithms']['RandomForestClassifier']["max_trees"]+1),
                  'max_depth':np.arange(model_info['design_state_data']['algorithms']['RandomForestClassifier']["min_depth"],model_info['design_state_data']['algorithms']['RandomForestClassifier']["max_depth"]+1),
                  'min_samples_leaf':np.arange(model_info['design_state_data']['algorithms']['RandomForestClassifier']["min_samples_per_leaf_min_value"],model_info['design_state_data']['algorithms']['RandomForestClassifier']["min_samples_per_leaf_max_value"]+1)}

      cv_rf_model=GridSearchCV(rf_model, param_grid, scoring=None, n_jobs=model_info['design_state_data']["hyperparameters"]["parallelism"], refit=True, 
                              cv=model_info['design_state_data']["hyperparameters"]["num_of_folds"],
                              verbose=0, pre_dispatch='2*n_jobs', error_score=np.nan, return_train_score=False)


      rf_model.fit(xtrain,ytrain)
      pred_y=rf_model.predict(xtest)
      print(f"Random Forest classifier  accuracy is {accuracy_score(ytest,pred_y)}")

  elif model_instance['GBTClassifier']:
    gbc_model=GradientBoostingClassifier(loss='log_loss', learning_rate=0.1, n_estimators=5, subsample=1.0, criterion='friedman_mse', min_samples_split=2, 
                                            min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=model_info['design_state_data']['algorithms']['GBTClassifier']["max_depth"],
                                            min_impurity_decrease=0.0, init=None, random_state=1, 
                                            max_features=max_feature, verbose=0, max_leaf_nodes=None, warm_start=False, validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)

    param_grid={'subsample':np.arange(model_info['design_state_data']['algorithms']['GBTClassifier']["min_subsample"],model_info['design_state_data']['algorithms']['GBTClassifier']["max_subsample"]+1),
                'max_depth':np.arange(model_info['design_state_data']['algorithms']['GBTClassifier']["min_depth"],model_info['design_state_data']['algorithms']['GBTClassifier']["max_depth"]+1),
                'n_iter_no_change':np.arange(model_info['design_state_data']['algorithms']['GBTClassifier']["min_iter"],model_info['design_state_data']['algorithms']['GBTClassifier']["max_iter"]+1)}

    cv_gb_model=GridSearchCV(gbc_model, param_grid, scoring=None, n_jobs=model_info['design_state_data']["hyperparameters"]["parallelism"], refit=True, 
                            cv=model_info['design_state_data']["hyperparameters"]["num_of_folds"],
                            verbose=0, pre_dispatch='2*n_jobs', error_score=np.nan, return_train_score=False)

    cv_gb_model.fit(xtrain,ytrain)
    pred_y=cv_gb_model.predict(xtest)
    print(f"Gradient Boosted classifier  accuracy is {accuracy_score(ytest,pred_y)}")


  elif model_instance['GBTRegressor']:

    gbr_model=GradientBoostingRegressor(loss='squared_error', learning_rate=0.1, n_estimators=10, subsample=0.5, criterion='friedman_mse', min_samples_split=2,
                                                min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=model_info['design_state_data']['algorithms']['GBTRegressor']["max_depth"], min_impurity_decrease=0.0, init=None, random_state=1,
                                              max_features=None, alpha=0.9, verbose=0, max_leaf_nodes=None, warm_start=False, validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)


    param_grid={'subsample':np.arange(model_info['design_state_data']['algorithms']['GBTRegressor']["min_subsample"],model_info['design_state_data']['algorithms']['GBTRegressor']["max_subsample"]+1),
                'max_depth':np.arange(model_info['design_state_data']['algorithms']['GBTRegressor']["min_depth"],model_info['design_state_data']['algorithms']['GBTRegressor']["max_depth"]+1),
                'n_iter_no_change':np.arange(model_info['design_state_data']['algorithms']['GBTRegressor']["min_iter"],model_info['design_state_data']['algorithms']['GBTRegressor']["max_iter"]+1)}

    cv_gb_model=GridSearchCV(gbr_model, param_grid, scoring=None, n_jobs=model_info['design_state_data']["hyperparameters"]["parallelism"], refit=True, 
                            cv=model_info['design_state_data']["hyperparameters"]["num_of_folds"],
                            verbose=0, pre_dispatch='2*n_jobs', error_score=np.nan, return_train_score=False)

    cv_gb_model.fit(xtrain,ytrain)
    pred_y=cv_gb_model.predict(xtest)
    print(f"Gradient Boosted Regressor  mean squared error is {round(mean_squared_error(ytest,pred_y),2)}")

  elif model_instance['LinearRegression']:
    lr=LinearRegression(fit_intercept=True, normalize='deprecated', copy_X=True, n_jobs=None, positive=False)
    lr.fit(xtrain,ytrain)
    pred_y=lr.predict(xtest)
    print(f"Linear Regression  mean squared error is {round(mean_squared_error(ytest,pred_y),2)}")

  elif model_instance['LogisticRegression']:
    logr=LogisticRegression(penalty='l2',dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=1,
                      solver='lbfgs', max_iter=model_info['design_state_data']['algorithms']['LogisticRegression']["max_iter"],
                        multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)

    param_grid={'max_iter':np.arange(model_info['design_state_data']['algorithms']['LogisticRegression']["min_iter"],model_info['design_state_data']['algorithms']['LogisticRegression']["max_iter"]+1),
                }

    cv_logr_model=GridSearchCV(logr, param_grid, scoring=None, n_jobs=model_info['design_state_data']["hyperparameters"]["parallelism"], refit=True, 
                            cv=model_info['design_state_data']["hyperparameters"]["num_of_folds"],
                            verbose=0, pre_dispatch='2*n_jobs', error_score=np.nan, return_train_score=False)

    cv_logr_model.fit(xtrain,ytrain)
    pred_y=cv_logr_model.predict(xtest)
    print(f"Logistic Regression  accuracy is {accuracy_score(ytest,pred_y)}")

  elif model_instance['RidgeRegression']:
    rr=Ridge(alpha=1.0, fit_intercept=True, normalize='deprecated', copy_X=True, max_iter=None, 
          tol=0.001, solver='auto', positive=False, random_state=None)

    param_grid={'max_iter':np.arange(model_info['design_state_data']['algorithms']["RidgeRegression"]["min_iter"],model_info['design_state_data']['algorithms']["RidgeRegression"]["max_iter"]+1),
                }

    cv_rr_model=GridSearchCV(rr, param_grid, scoring=None, n_jobs=model_info['design_state_data']["hyperparameters"]["parallelism"], refit=True, 
                            cv=model_info['design_state_data']["hyperparameters"]["num_of_folds"],
                            verbose=0, pre_dispatch='2*n_jobs', error_score=np.nan, return_train_score=False)

    cv_rr_model.fit(xtrain,ytrain)
    pred_y=cv_rr_model.predict(xtest)

    print(f"Ridge Regressor  mean squared error is {round(mean_squared_error(ytest,pred_y),2)}")

  elif model_instance['LassoRegression']:
    lr=Lasso(alpha=1.0,fit_intercept=True, normalize='deprecated', precompute=False, copy_X=True, max_iter=1000, tol=0.0001, 
                            warm_start=False, positive=False, random_state=None, selection='cyclic')

    param_grid={'max_iter':np.arange(model_info['design_state_data']['algorithms']["LassoRegression"]["min_iter"],model_info['design_state_data']['algorithms']["LassoRegression"]["max_iter"]+1),
                }


    cv_lr_model=GridSearchCV(lr, param_grid, scoring=None, n_jobs=model_info['design_state_data']["hyperparameters"]["parallelism"], refit=True, 
                            cv=model_info['design_state_data']["hyperparameters"]["num_of_folds"],
                            verbose=0, pre_dispatch='2*n_jobs', error_score=np.nan, return_train_score=False)

    cv_lr_model.fit(xtrain,ytrain)
    pred_y=cv_lr_model.predict(xtest)
    print(f"Lasso Regressor  mean squared error is {round(mean_squared_error(ytest,pred_y),2)}")

  elif model_instance['DecisionTreeRegressor']:
    dtr=DecisionTreeRegressor(criterion='squared_error', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, 
                              min_weight_fraction_leaf=0.0, max_features=max_feature, random_state=1, max_leaf_nodes=None, 
                              min_impurity_decrease=0.0, ccp_alpha=0.0)


    param_grid={'max_depth':np.arange(model_info['design_state_data']['algorithms']["DecisionTreeRegressor"]["min_depth"],model_info['design_state_data']['algorithms']["DecisionTreeRegressor"]["max_depth"]+1)}

    cv_dtr_model=GridSearchCV(dtr, param_grid, scoring=None, n_jobs=model_info['design_state_data']["hyperparameters"]["parallelism"], refit=True, 
                            cv=model_info['design_state_data']["hyperparameters"]["num_of_folds"],
                            verbose=0, pre_dispatch='2*n_jobs', error_score=np.nan, return_train_score=False)

    cv_dtr_model.fit(xtrain,ytrain)
    pred_y=cv_dtr_model.predict(xtest)
    print(f"Decision Tree Regression  mean square error is {round(mean_squared_error(ytest,pred_y),2)}")

  elif model_instance['DecisionTreeClassifier']:
    if model_info['design_state_data']['algorithms']["DecisionTreeClassifier"]["use_gini"]==False:
      criterion='entropy'
    else:
      criterion='gini'

    dtc=DecisionTreeClassifier(criterion=criterion, splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1,
                                        min_weight_fraction_leaf=0.0, max_features=max_feature, random_state=1, max_leaf_nodes=None, min_impurity_decrease=0.0, 
                                        class_weight=None, ccp_alpha=0.0)

    param_grid={'max_depth':np.arange(model_info['design_state_data']['algorithms']["DecisionTreeClassifier"]["min_depth"],model_info['design_state_data']['algorithms']["DecisionTreeClassifier"]["max_depth"]+1)}

    cv_dtc_model=GridSearchCV(dtc, param_grid, scoring=None, n_jobs=model_info['design_state_data']["hyperparameters"]["parallelism"], refit=True, 
                            cv=model_info['design_state_data']["hyperparameters"]["num_of_folds"],
                            verbose=0, pre_dispatch='2*n_jobs', error_score=np.nan, return_train_score=False)

    cv_dtc_model.fit(xtrain,ytrain)
    pred_y=cv_dtc_model.predict(xtest)
    print(f"Decision Tree classifier  accuracy is {accuracy_score(ytest,pred_y)}")

  elif model_instance['SGD']:
    sgd=SGDRegressor(loss='squared_error',penalty='l2', alpha=model_info['design_state_data']['algorithms']['SGD']["alpha_value"][0], l1_ratio=0.15,
                                  fit_intercept=True, max_iter=1000, tol=model_info['design_state_data']['algorithms']['SGD']["tolerance"], 
                                  shuffle=True, verbose=0, epsilon=0.1, random_state=None, learning_rate='invscaling', eta0=0.01, power_t=0.25, 
                                  early_stopping=False, validation_fraction=0.1, n_iter_no_change=5, warm_start=False, average=False)

    sgd.fit(xtrain,ytrain)
    pred_y=sgd.predict(xtest)
    print(f"SGD Regression  mean squared error is {round(mean_squared_error(ytest,pred_y),2)}")
  return 1



model_prediction(df,model_info)


Random Forest Regressor  mean squared error is 0.03


1